## Disclaimer & Copyright

Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Gemini - Image analysis and software development with ER-Diagram
Feedback : shins777@gmail.com.

* This notebook explain about how to use Gemini to understand image in multimodality features of Gemini.  
* Multimodality is one of the core capabilities in the latest foundation models.
* Gemini models are fundamentally built on this multimodality.

* In this demo, we are able to understand how to leverage the multimodality to improve the productivity of software development.

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 12.2 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다.
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set the environment on GCP Project
* Configure project information
* Recommend to use the latest model.

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Multimodality basic code

In [7]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data

def generate(file_path:str, query:str, mime_type:str):

    pdf_obj = get_obj_content(obj_loc=file_path)
    doc_obj = Part.from_data(data=base64.b64decode(pdf_obj), mime_type=mime_type)

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [doc_obj, query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

In [8]:
root_path = "/content/drive/MyDrive/projects"
file_path = "/google_gen_ai_sample/contents/images/ERD.jpeg"

path = root_path + file_path
mime_type = "image/jpeg"

prompt = """You are an AI assistant helping software development.
Write down python code in the following instruction.

1. Refer to the ERD to write a code.
2. Retrieve the result from joining tables in the ERD.
3. Get the information customer id, delivery id, payment id by customer id.

"""

outcome = generate(path, prompt, mime_type)

display(Markdown(outcome))



```python
import sqlite3

# Connect to the database
conn = sqlite3.connect('pizza.db')

# Create a cursor object
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE Customer (
        CustomerID INTEGER PRIMARY KEY,
        Fname TEXT,
        Lname TEXT,
        Address TEXT,
        Phone TEXT,
        E_mail TEXT
    )
''')

cursor.execute('''
    CREATE TABLE OrderHeader (
        OrderID INTEGER PRIMARY KEY,
        OrderDate DATE,
        OrderTime TIME,
        CustomerID INTEGER,
        FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
    )
''')

cursor.execute('''
    CREATE TABLE OrderLine (
        LineID INTEGER PRIMARY KEY,
        OrderID INTEGER,
        PizzaID INTEGER,
        Quantity INTEGER,
        FOREIGN KEY (OrderID) REFERENCES OrderHeader(OrderID),
        FOREIGN KEY (PizzaID) REFERENCES Pizza(PizzaID)
    )
''')

cursor.execute('''
    CREATE TABLE Pizza (
        PizzaID INTEGER PRIMARY KEY,
        Name TEXT,
        Specification TEXT,
        Price REAL
    )
''')

cursor.execute('''
    CREATE TABLE Delivery (
        DeilveryID INTEGER PRIMARY KEY,
        OrderID INTEGER,
        Type TEXT,
        Status TEXT,
        Departure TEXT,
        Arrival TEXT,
        FOREIGN KEY (OrderID) REFERENCES OrderHeader(OrderID)
    )
''')

cursor.execute('''
    CREATE TABLE Payment (
        PaymentID INTEGER PRIMARY KEY,
        PDate DATE,
        Type TEXT,
        OrderID INTEGER,
        CustomerID INTEGER,
        Total REAL,
        FOREIGN KEY (OrderID) REFERENCES OrderHeader(OrderID),
        FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
    )
''')

# Insert some data into the tables
cursor.execute("INSERT INTO Customer VALUES (1, 'John', 'Doe', '123 Main St', '555-1212', 'john.doe@email.com')")
cursor.execute("INSERT INTO OrderHeader VALUES (1, '2023-03-08', '12:00:00', 1)")
cursor.execute("INSERT INTO OrderLine VALUES (1, 1, 1, 2)")
cursor.execute("INSERT INTO Pizza VALUES (1, 'Pepperoni', 'Classic pepperoni pizza', 12.99)")
cursor.execute("INSERT INTO Delivery VALUES (1, 1, 'Delivery', 'Delivered', '2023-03-08 12:00:00', '2023-03-08 12:30:00')")
cursor.execute("INSERT INTO Payment VALUES (1, '2023-03-08', 'Credit Card', 1, 1, 25.98)")

# Retrieve the result from joining tables in the ERD
cursor.execute('''
    SELECT
        c.CustomerID,
        d.DeilveryID,
        p.PaymentID
    FROM Customer c
    JOIN OrderHeader oh ON c.CustomerID = oh.CustomerID
    JOIN Delivery d ON oh.OrderID = d.OrderID
    JOIN Payment p ON oh.OrderID = p.OrderID
''')

# Get the information customer id, delivery id, payment id by customer id
rows = cursor.fetchall()
for row in rows:
    print(f"Customer ID: {row[0]}, Delivery ID: {row[1]}, Payment ID: {row[2]}")

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()
```